In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()
with open("../login.json", "r") as j:
    misDatos = json.load(j)
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [2]:
query_empresacontactoconvenio = (
    "select Id,Paciente__r.IdentificationId__pc, Empresa__r.P_liza__c, Relaci_n_Con_Asegurado__c,N_Documento_Titular__c,Empresa__r.Rut_Empresa__c,Empresa__c from EmpresaContactoConvenios__c "
)
empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
rut_yidEmCoCo = pd.DataFrame()
rut_yidEmCoCo["Id empreconcon"] = empresacontactoconvenio["Id"]
rut_yidEmCoCo["rut"] =  empresacontactoconvenio["Paciente__r.IdentificationId__pc"]
rut_yidEmCoCo["Relacion"] = empresacontactoconvenio["Relaci_n_Con_Asegurado__c"] 
rut_yidEmCoCo["rutT"] = empresacontactoconvenio["N_Documento_Titular__c"]
rut_yidEmCoCo["poliza"] = empresacontactoconvenio["Empresa__r.P_liza__c"]
rut_yidEmCoCo["rut_E"] = empresacontactoconvenio["Empresa__r.Rut_Empresa__c"]
rut_yidEmCoCo["empresa"] = empresacontactoconvenio["Empresa__c"]
rut_yidEmCoCo = rut_yidEmCoCo.drop_duplicates(["rut","rutT","empresa","Relacion"], keep="last")
rut_yidEmCoCo.head()

empresacontactoconvenio["RutEstaoNo"] = empresacontactoconvenio["Id"].isin(rut_yidEmCoCo["Id empreconcon"])
noestan = empresacontactoconvenio[(empresacontactoconvenio["RutEstaoNo"] == False)]


In [ ]:
noestan.to_excel("F:/Empresas SF/reportes/repetidos_general.xlsx", index=False)